# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

if api_key and api_key.startswith('AIza') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gemini-3-pro-preview'
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini = OpenAI(base_url=GEMINI_BASE_URL, api_key=api_key)

API key looks good so far


In [3]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/proficient/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://edwarddonner.com/curriculum/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/11/11/ai-live-event/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/1

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [4]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/curriculum/
https://edwarddonner.com/proficient/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://edwarddonner.com/curriculum/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/
https://edwarddonner.com/2026/01/04/ai-builder-with-n8n-create-agents-and-voice-agents/

In [9]:
def select_relevant_links(url):
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [10]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum page', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'}]}

In [11]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [12]:
select_relevant_links("https://edwarddonner.com")

Selecting relevant links for https://edwarddonner.com by calling gemini-3-pro-preview
Found 11 relevant links


{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'curriculum', 'url': 'https://edwarddonner.com/curriculum/'},
  {'type': 'portfolio project', 'url': 'https://edwarddonner.com/proficient/'},
  {'type': 'portfolio project',
   'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'portfolio project', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'associated company',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter profile', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook profile',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [13]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-3-pro-preview
Found 7 relevant links


{'links': [{'type': 'enterprise page',
   'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company profile', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'brand assets', 'url': 'https://huggingface.co/brand'},
  {'type': 'linkedin',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [14]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [15]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gemini-3-pro-preview
Found 15 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 2M+ models
Trending on
this week
Models
moonshotai/Kimi-K2.5
Updated
about 21 hours ago
•
123k
•
1.6k
tencent/HunyuanImage-3.0-Instruct
Updated
about 15 hours ago
•
194
•
805
zai-org/GLM-OCR
Updated
1 day ago
•
2.26k
•
458
Tongyi-MAI/Z-Image
Updated
7 days ago
•
8.28k
•
831
stepfun-ai/Step-3.5-Flash
Updated
about 19 hours ago
•
1.83k
•
391
Browse 2M+ models
Spaces
Running
on
Zero
Featured
1.19k
Qwen3-TTS Demo
🎙
1.19k
Generate realistic speech from text with custom voices or voice cloning
Running
on
Zero
MCP
1.98k
Z Image Turbo
🖼
1.98k
Generate stunning images fro

In [16]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-3-pro-preview
Found 14 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 2M+ models\nTrending on\nthis week\nModels\nmoonshotai/Kimi-K2.5\nUpdated\nabout 21 hours ago\n•\n123k\n•\n1.6k\ntencent/HunyuanImage-3.0-Instruct\nUpdated\nabout 15 hours ago\n•\n194\n•\n805\nzai-org/GLM-OCR\nUpdated\n1 day ago\n•\n2.26k\n•\n458\nTongyi-MAI/Z-Image\nUpdated\n7 days ago\n•\n8.28k\n•\n831\nstepfun-ai/Step-3.5-Flash\nUpdated\nabout 19 hours ago\n•\n1.83k\n•\n391\nBrowse 2M+ models\nSpaces\nRunning\non\nZero\nFeatured\n

In [19]:
def create_brochure(company_name, url):
    response = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-3-pro-preview
Found 12 relevant links


# Hugging Face
**The AI Community Building the Future**

### The Home of Machine Learning
Hugging Face is the collaboration platform where the machine learning community creates, discovers, and builds the future of AI. Often referred to as the "GitHub of Machine Learning," Hugging Face has established itself as the central hub for the global AI revolution, providing the open-source stack that allows developers to move faster and build better.

---

### By The Numbers
The scale of the Hugging Face ecosystem is unmatched, serving as the default repository for the world's most advanced AI assets:
*   **2,000,000+ Models:** From Large Language Models to specialized tools.
*   **500,000+ Datasets:** The fuel for training the next generation of AI.
*   **1,000,000+ Applications:** "Spaces" where users demo and run AI apps instantly.

---

### For Enterprise & Teams
**Scale your organization with the world’s leading AI platform.**

Hugging Face provides a dedicated Enterprise Hub designed to give organizations the most advanced platform to build AI with enterprise-grade security and compliance.

**Key Enterprise Features:**
*   **Security & Control:** Connect securely to your identity provider with Single Sign-On (SSO) integration.
*   **Governance:** comprehensive audit logs that report on all actions taken within your repository.
*   **Data Sovereignty:** Manage and audit the specific regions where your repository data is located.
*   **Flexible Pricing:**
    *   **Team Plan:** Starts at $20/user/month for small groups.
    *   **Enterprise Plan:** Flexible contracts for large-scale organizations requiring advanced access controls and dedicated support.

---

### The Ecosystem
Hugging Face is not limited to text; it is the playground for **multimodal AI**. The platform hosts and computes across all realities:
*   **Text & NLP**
*   **Computer Vision & Image Generation**
*   **Audio & Speech Synthesis**
*   **Video Generation**
*   **3D Modeling**

Through **Spaces**, developers can showcase their work (demos) running on the "Zero" GPU infrastructure or upgrade to paid compute for high-performance needs.

---

### Culture & Community
**"Collaborate on ML better."**

At its heart, Hugging Face is about democratizing Artificial Intelligence. The company culture is deeply rooted in **Open Source** ideals. It is a place where:
*   **Collaboration is King:** Users host and collaborate on unlimited public models and datasets.
*   **Meritocracy Prevails:** Developers build their portfolios and ML profiles by sharing their work with the world.
*   **Innovation Moves Fast:** The community updates trending models (like Qwen, MoonshotAI, and HunyuanImage) daily, ensuring the platform is always at the bleeding edge of technology.

### Join the Revolution
Whether you are a developer looking to host your first model, an enterprise seeking to integrate AI securely, or an investor looking at the infrastructure layer of the AI economy, Hugging Face is the engine room of the industry.

**Explore. Collaborate. Build.**
*Visit us at huggingface.co*

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [23]:
def stream_brochure(company_name, url):
    stream = gemini.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gemini-3-pro-preview
Found 14 relevant links


# Hugging Face: The AI Community Building the Future

**The Home of Machine Learning**

Welcome to Hugging Face, the world's leading platform where the machine learning community collaborates on models, datasets, and applications. We are the central hub for the open-source AI revolution, empowering creators, researchers, and organizations to create, discover, and collaborate on ML better and faster.

---

### 🚀 For Prospective Customers & Users

Whether you are a solo developer, a research lab, or a global enterprise, Hugging Face provides the infrastructure to democratize AI.

**The Ecosystem**
Our platform hosts the largest collection of open-source resources in the industry, covering all modalities including text, image, video, audio, and 3D.
*   **Models:** Browse and deploy over **2 Million+ models**.
*   **Datasets:** Access **500k+ datasets** to train and fine-tune your systems.
*   **Spaces:** Explore **1 Million+ applications**. Use Spaces to host demos and showcase your ML portfolio instantly.

**Enterprise-Grade Solutions**
Scale your organization with our **Enterprise Hub**. We provide the most advanced platform to build AI with the security and control large teams require.
*   **Security & Compliance:** Single Sign-On (SSO), comprehensive audit logs, and regional data control.
*   **Management:** Granular access controls via Resource Groups and centralized token management.
*   **Compute:** Access advanced compute options, including ZeroGPU, to accelerate your workflows.
*   **Private Collaboration:** Secure 1TB of private storage per member and a Private Datasets Viewer for internal collaboration.

---

### 📈 For Investors

Hugging Face has established itself as the fundamental infrastructure layer for the AI boom. We are not just a tool; we are the network effect of Artificial Intelligence.

*   **Unrivaled Traction:** We host millions of models and applications, serving as the de facto "GitHub" for machine learning.
*   **Monetization Strategy:** We offer a tiered business model ranging from individual "Pro" subscriptions to scalable "Team" plans and high-value "Enterprise" contracts.
*   **Market Position:** We are the home of the "Open Source Stack," positioning us at the center of innovation for the most forward-thinking AI organizations globally.

---

### 💼 For Recruits: Careers & Culture

**Build the Future with Us**
At Hugging Face, we are more than a tech company; we are a community movement. We believe in the power of open collaboration to advance artificial intelligence.

**Our Culture**
*   **Collaborative:** We build tools that allow the world to work together. Collaboration is in our DNA.
*   **Open Source First:** We are dedicated to democratizing AI, ensuring technology is accessible, transparent, and shared.
*   **Innovative:** From "ZeroGPU" technology to supporting the latest multimodal models, we work on the bleeding edge of what is possible.

**Join the Team**
We are looking for passionate individuals ready to shape the future of machine learning. While specific roles evolve quickly, we are always seeking talent to help us manage our growing infrastructure, improve our community features, and support our enterprise partners.

---

### 💳 Pricing & Plans

**Pro Account ($9/month)**
*   For individuals wanting to boost their profile.
*   10x private storage, higher GPU priority, and exclusive badges.

**Team Plan (Starts at $20/user/month)**
*   Instant setup for growing organizations.
*   Includes private workspaces and team management tools.

**Enterprise (Custom)**
*   Tailored for large-scale organizations requiring advanced security, SSO, and priority support.

---

**Ready to start?**
Join the community today and begin building your portfolio.
**Visit us at Hugging Face**

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>